In [ ]:
# 1. Cài đặt các thư viện cần thiết trên Kaggle
!pip install transformers torch pandas numpy rouge-score bert-score huggingface_hub

# 2. Đăng nhập vào Hugging Face để tải mô hình
from huggingface_hub import login

# Dán API Token của bạn (lấy từ Hugging Face Settings > Access Tokens)
api_token = ""  # Thay bằng API Token của bạn
login(api_token)

# 3. Tải mô hình và tokenizer từ Hugging Face
from transformers import BartTokenizer, BartForConditionalGeneration, GenerationConfig

repo_id = "HTThuanHcmus/bart-finetuned-scientific"  # Thay bằng username và tên repo của bạn
tokenizer = BartTokenizer.from_pretrained(repo_id)
model = BartForConditionalGeneration.from_pretrained(repo_id)

# Tạo GenerationConfig thủ công để tránh lỗi
generation_config = GenerationConfig(
    early_stopping=True,
    max_length=32,
    num_beams=5
)
model.generation_config = generation_config

# 4. Hàm dự đoán tiêu đề từ abstract bằng BART
def predict_title_bart(abstract):
    inputs = tokenizer(abstract, max_length=512, truncation=True, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    outputs = model.generate(
        **inputs,
        max_length=32,
        num_beams=5,
        early_stopping=True,
        decoder_start_token_id=tokenizer.bos_token_id if tokenizer.bos_token_id is not None else tokenizer.convert_tokens_to_ids('<s>')
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 5. Tải dữ liệu test trên Kaggle
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer
from bert_score import score

# Giả sử dữ liệu của bạn nằm trong /kaggle/input
# File CSV chứa các cột: abstract, title (tiêu đề thực tế), llm_title (tiêu đề từ LLM)
data_df = pd.read_csv('/kaggle/input/compare-bart-llm/random_500_samples_full.csv')  # Thay bằng đường dẫn thực tế trên Kaggle
abstracts = data_df['abstract'].tolist()
true_titles = data_df['title'].tolist()
llm_titles = data_df['title-llm'].tolist()

# 6. Dự đoán tiêu đề bằng BART
bart_titles = [predict_title_bart(abs) for abs in abstracts]

# 7. Tính ROUGE và BERTScore cho cả hai tập tiêu đề
# ROUGE
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)

# ROUGE cho LLM
llm_rouge1_scores = []
llm_rouge2_scores = []
llm_rougeL_scores = []
llm_rougeLsum_scores = []

for pred, ref in zip(llm_titles, true_titles):
    scores = scorer.score(ref, pred)
    llm_rouge1_scores.append(scores['rouge1'].fmeasure)
    llm_rouge2_scores.append(scores['rouge2'].fmeasure)
    llm_rougeL_scores.append(scores['rougeL'].fmeasure)
    llm_rougeLsum_scores.append(scores['rougeLsum'].fmeasure)

# ROUGE cho BART
bart_rouge1_scores = []
bart_rouge2_scores = []
bart_rougeL_scores = []
bart_rougeLsum_scores = []

for pred, ref in zip(bart_titles, true_titles):
    scores = scorer.score(ref, pred)
    bart_rouge1_scores.append(scores['rouge1'].fmeasure)
    bart_rouge2_scores.append(scores['rouge2'].fmeasure)
    bart_rougeL_scores.append(scores['rougeL'].fmeasure)
    bart_rougeLsum_scores.append(scores['rougeLsum'].fmeasure)

# BERTScore cho LLM
llm_P, llm_R, llm_F1 = score(llm_titles, true_titles, lang="en", verbose=True)
llm_bert_precision_scores = llm_P.tolist()
llm_bert_recall_scores = llm_R.tolist()
llm_bert_f1_scores = llm_F1.tolist()

# BERTScore cho BART
bart_P, bart_R, bart_F1 = score(bart_titles, true_titles, lang="en", verbose=True)
bart_bert_precision_scores = bart_P.tolist()
bart_bert_recall_scores = bart_R.tolist()
bart_bert_f1_scores = bart_F1.tolist()

# 8. So sánh trung bình ROUGE và BERTScore
print("So sánh trung bình ROUGE và BERTScore:\n")

print("LLM Titles:")
print(f"ROUGE-1: {np.mean(llm_rouge1_scores):.4f}")
print(f"ROUGE-2: {np.mean(llm_rouge2_scores):.4f}")
print(f"ROUGE-L: {np.mean(llm_rougeL_scores):.4f}")
print(f"ROUGE-Lsum: {np.mean(llm_rougeLsum_scores):.4f}")
print(f"BERTScore Precision: {np.mean(llm_bert_precision_scores):.4f}")
print(f"BERTScore Recall: {np.mean(llm_bert_recall_scores):.4f}")
print(f"BERTScore F1: {np.mean(llm_bert_f1_scores):.4f}\n")

print("BART Fine-tuned Titles:")
print(f"ROUGE-1: {np.mean(bart_rouge1_scores):.4f}")
print(f"ROUGE-2: {np.mean(bart_rouge2_scores):.4f}")
print(f"ROUGE-L: {np.mean(bart_rougeL_scores):.4f}")
print(f"ROUGE-Lsum: {np.mean(bart_rougeLsum_scores):.4f}")
print(f"BERTScore Precision: {np.mean(bart_bert_precision_scores):.4f}")
print(f"BERTScore Recall: {np.mean(bart_bert_recall_scores):.4f}")
print(f"BERTScore F1: {np.mean(bart_bert_f1_scores):.4f}\n")

# 9. Phân tích chi tiết: Số lượng tiêu đề mà BART vượt trội hơn LLM và ngược lại
bart_better_rouge1 = sum(1 for llm_score, bart_score in zip(llm_rouge1_scores, bart_rouge1_scores) if bart_score > llm_score)
llm_better_rouge1 = sum(1 for llm_score, bart_score in zip(llm_rouge1_scores, bart_rouge1_scores) if llm_score > bart_score)

bart_better_bertscore = sum(1 for llm_score, bart_score in zip(llm_bert_f1_scores, bart_bert_f1_scores) if bart_score > llm_score)
llm_better_bertscore = sum(1 for llm_score, bart_score in zip(llm_bert_f1_scores, bart_bert_f1_scores) if llm_score > bart_score)

print("Phân tích chi tiết:")
print(f"Số tiêu đề mà BART có ROUGE-1 cao hơn LLM: {bart_better_rouge1}")
print(f"Số tiêu đề mà LLM có ROUGE-1 cao hơn BART: {llm_better_rouge1}")
print(f"Số tiêu đề mà BART có BERTScore F1 cao hơn LLM: {bart_better_bertscore}")
print(f"Số tiêu đề mà LLM có BERTScore F1 cao hơn BART: {llm_better_bertscore}\n")

# 10. Lưu kết quả chi tiết vào file CSV
results_df = pd.DataFrame({
    'abstract': abstracts,
    'true_title': true_titles,
    'llm_title': llm_titles,
    'bart_title': bart_titles,
    'llm_rouge1': llm_rouge1_scores,
    'llm_rouge2': llm_rouge2_scores,
    'llm_rougeL': llm_rougeL_scores,
    'llm_rougeLsum': llm_rougeLsum_scores,
    'llm_bertscore_precision': llm_bert_precision_scores,
    'llm_bertscore_recall': llm_bert_recall_scores,
    'llm_bertscore_f1': llm_bert_f1_scores,
    'bart_rouge1': bart_rouge1_scores,
    'bart_rouge2': bart_rouge2_scores,
    'bart_rougeL': bart_rougeL_scores,
    'bart_rougeLsum': bart_rougeLsum_scores,
    'bart_bertscore_precision': bart_bert_precision_scores,
    'bart_bertscore_recall': bart_bert_recall_scores,
    'bart_bertscore_f1': bart_bert_f1_scores
})
results_df.to_csv('/kaggle/working/comparison_results.csv', index=False)
print("Đã lưu kết quả chi tiết vào '/kaggle/working/comparison_results.csv'")